In [67]:
import numpy as np
import pandas as pd
import itertools
import copy

from scipy.optimize import fsolve
from scipy.special import gamma

In [68]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))

In [69]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [70]:
def gen_call_arrival_time():
    return np.random.exponential(1)

In [71]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [193]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return (scale * np.random.weibull(shape))[0]

In [194]:
class Call(object):
    
    def __init__(self, call_index, location, time_arrival, interarrival_time, priority = 0):
        
        self.call_index = call_index
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.arrival_time = time_arrival
        self.next_arrival_time = self.arrival_time + interarrival_time
        self.priority = priority

In [195]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [196]:
def get_ambulance_travel_time(distance, speed = 10):
    return distance/speed

In [197]:
class Ambulance(object):
    
    def __init__(self, base):
        
        #fix the ambulance's home base
        self.base = base
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        self.call = -1 # -1 if not assigned to any call
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self):
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, self.base)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(5, self.origin, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        self.call = -1
        
    def return_to_base(self):
        self.update_status(0, copy.deepcopy(self.destination), copy.deepcopy(self.endtime), np.inf)
        
        
        
    def assign_to_call(self, call, time, index):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        self.call = call.call_index # updated assigned call index
        
        call.status = index
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.pop(call.call_index)
    
    def transport_to_hospital(self, city):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = city.width + city.height
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [198]:
def arrival(call_index, ambulanceList, callList, time_arrival, M, timeline):
    
    call = Call(call_index, gen_call_arrival_loc(city), time_arrival, gen_call_arrival_time())
    
    if len(callList) >= M:
        i = timeline.shape[0]
        timeline.loc[i] = [call_index, '', 7, time_arrival]
        print("New call arrived. No more capacity. Reject call.")
    
    else:
    
        print("New call arrived. Add to call list.")
#         callList.append(call)
        callList[call_index] = call

        min_distance = 15
        assign = -1
        index = 0

        nearest_distance = 15
        for ambulance in ambulanceList:

            if ambulance.status == 0:
                distance = get_distance(ambulance.origin, call.location)
                if distance < nearest_distance:
                    nearest_distance = distance
                    assign = index

            index = index + 1

        if assign > -1:
            # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
            print("Idle ambulance at base available. Assign. Now travelling to call location.")
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, assign, 1, time_arrival]
            ambulanceList[assign].assign_to_call(call, time_arrival, assign)

        else:
            print("No available ambulance at the moment.")
            
    time_arrival = call.next_arrival_time
    
    return ambulanceList, callList, time_arrival, timeline

In [199]:
def get_next_event(time_arrival, ambulanceList, callList, city, M, timeline):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = copy.deepcopy(ambulance.endtime)
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    print(time_arrival, ambulanceEndTime_min, next_event_time)
    if next_event_time == time_arrival:
        print("New call arrived.")
        i = timeline.shape[0]
        all_call = set(timeline['Call'])
        call_index = len(all_call) if '' not in all_call else len(all_call)-1
        timeline.loc[i] = [call_index, '', 0, time_arrival]
        ambulanceList, callList, time_arrival, timeline = arrival(call_index, ambulanceList, callList, time_arrival, M, timeline)
        
    else:
        if ambulanceList[index_min].status == 1:
            print("Now reach call location. Start at-location treatment. Remove call from call list.")
            call_index = ambulanceList[index_min].call
            call = callList[call_index]
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 2, next_event_time]
            ambulanceList[index_min].reach_call_location(call, callList)
            
        elif ambulanceList[index_min].status == 2:
            print("Now finish at-location treatment. Start going to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 3, next_event_time]
            ambulanceList[index_min].transport_to_hospital(city)
            
        elif ambulanceList[index_min].status == 3:
            print("Now reach hospital. Start transferring patient to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 4, next_event_time]
            ambulanceList[index_min].reach_hospital()
            
        elif ambulanceList[index_min].status == 4:
            
            print("Now finish transfering patient to hospital. Decide next step (assign to call or return to base).")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 5, next_event_time]
            
            if len(callList) == 0:
                print("Return to base.")
                ambulanceList[index_min].redployment()
            else:
                print("Call waiting. Assign to the first unassigned call in queue.")
                assign = -1
                call_index = min(callList.keys())
                index = 0
                while assign == -1 and index < M:

                    if callList[call_index].status == -1:
                        assign = call_index
                        
                    call_index += 1
                    index = index + 1
                
                if index == M:
                    # calls in callList have all been assigned with an ambulance
                    ambulanceList[index_min].redployment()
                else:
                    i = timeline.shape[0]
                    timeline.loc[i] = [assign, index_min, 1, next_event_time]
                    print([assign, index_min, 1, next_event_time])
                    ambulanceList[index_min].assign_to_call(callList[assign], next_event_time, index_min)
                    
        elif ambulanceList[index_min].status == 5:
            i = timeline.shape[0]
            timeline.loc[i] = ['', index_min, 6, next_event_time]
            print("Now reployed ambulance reach base. Start idling.")
            ambulanceList[index_min].return_to_base()
    
    return time_arrival, ambulanceList, callList, timeline

In [200]:
def print_ambulance(ambulanceList):
    statusList = []
    originList = []
    destinationList = []
    timeList = []
    endtimeList = []
    
    for ambulance in ambulanceList:
        statusList.append(ambulance.status)
        originList.append(ambulance.origin)
        destinationList.append(ambulance.destination)
        timeList.append(ambulance.time)
        endtimeList.append(ambulance.endtime)
    
    return statusList, originList, destinationList, timeList, endtimeList
    

In [201]:
def print_call(callList):
    
    statusList = []
    locationList = []
    timeList = []
    
    for call_index in callList:
        call = callList[call_index]
        statusList.append(call.status)
        locationList.append(call.location)
        timeList.append(call.arrival_time)
    
    return statusList, locationList, timeList

# Set up the City

In [202]:
city = City(10, 5, 2, 4)
ambulanceList = [Ambulance(city.base_location[0]), Ambulance(city.base_location[1])]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

# Repetitively run the following two blocks to visualize the process

In [203]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

print("Next Arrival: ", time_arrival)

print("Ambulance List Print: ")
print("Status List: ", am_statusList)
print("Origin List: ", am_originList)
print("Destination List: ", am_destinationList)
print("Event start time List: ", am_timeList)
print("Event end time List: ", am_endtimeList)

print("Call List Print: ")
print("Status List: ", cl_statusList)
print("Location List: ", cl_locationList)
print("Arrival time List: ", cl_timeList)

0 inf 0
New call arrived.
New call arrived. Add to call list.
Idle ambulance at base available. Assign. Now travelling to call location.
Next Arrival:  0.2564812962329283
Ambulance List Print: 
Status List:  [1, 0]
Origin List:  [(3.3333333333333335, 1.6666666666666667), (3.3333333333333335, 3.3333333333333335)]
Destination List:  [(9.42947450043328, 0.7459981649967379), (3.3333333333333335, 3.3333333333333335)]
Event start time List:  [0, 0]
Event end time List:  [0.7016809668769874, inf]
Call List Print: 
Status List:  [0]
Location List:  [(9.42947450043328, 0.7459981649967379)]
Arrival time List:  [0]


In [204]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0


In [205]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

print("Next Arrival: ", time_arrival)

print("Ambulance List Print: ")
print("Status List: ", am_statusList)
print("Origin List: ", am_originList)
print("Destination List: ", am_destinationList)
print("Event start time List: ", am_timeList)
print("Event end time List: ", am_endtimeList)

print("Call List Print: ")
print("Status List: ", cl_statusList)
print("Location List: ", cl_locationList)
print("Arrival time List: ", cl_timeList)

0.2564812962329283 0.7016809668769874 0.2564812962329283
New call arrived.
New call arrived. Add to call list.
Idle ambulance at base available. Assign. Now travelling to call location.
Next Arrival:  3.0189310636067708
Ambulance List Print: 
Status List:  [1, 1]
Origin List:  [(3.3333333333333335, 1.6666666666666667), (3.3333333333333335, 3.3333333333333335)]
Destination List:  [(9.42947450043328, 0.7459981649967379), (3.098187002278734, 4.117361510766165)]
Event start time List:  [0, 0.2564812962329283]
Event end time List:  [0.7016809668769874, 0.3583987470816714]
Call List Print: 
Status List:  [0, 1]
Location List:  [(9.42947450043328, 0.7459981649967379), (3.098187002278734, 4.117361510766165)]
Arrival time List:  [0, 0.2564812962329283]


In [206]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

3.0189310636067708 0.3583987470816714 0.3583987470816714
Now reach call location. Start at-location treatment. Remove call from call list.


In [207]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399


In [208]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

3.0189310636067708 0.7016809668769874 0.7016809668769874
Now reach call location. Start at-location treatment. Remove call from call list.


In [209]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

3.0189310636067708 6.844214230297116 3.0189310636067708
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.


In [210]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

3.7040497768826794 6.844214230297116 3.7040497768826794
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.


In [211]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399
5,0,0,2,0.701681
6,2,,0,3.01893
7,3,,0,3.70405


In [212]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

3.84670878501578 6.844214230297116 3.84670878501578
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.


In [213]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399
5,0,0,2,0.701681
6,2,,0,3.01893
7,3,,0,3.70405
8,4,,0,3.84671


In [214]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

5.9194444864167775 6.844214230297116 5.9194444864167775
New call arrived.
New call arrived. No more capacity. Reject call.


In [215]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399
5,0,0,2,0.701681
6,2,,0,3.01893
7,3,,0,3.70405
8,4,,0,3.84671
9,5,,0,5.91944


In [216]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

7.5955653442470945 6.844214230297116 6.844214230297116
Now finish at-location treatment. Start going to hospital.


In [217]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399
5,0,0,2,0.701681
6,2,,0,3.01893
7,3,,0,3.70405
8,4,,0,3.84671
9,5,,0,5.91944


In [218]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

7.5955653442470945 7.2958951971741035 7.2958951971741035
Now reach hospital. Start transferring patient to hospital.


In [219]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399
5,0,0,2,0.701681
6,2,,0,3.01893
7,3,,0,3.70405
8,4,,0,3.84671
9,5,,0,5.91944


In [220]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

7.5955653442470945 9.232500014637587 7.5955653442470945
New call arrived.
New call arrived. No more capacity. Reject call.


In [221]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399
5,0,0,2,0.701681
6,2,,0,3.01893
7,3,,0,3.70405
8,4,,0,3.84671
9,5,,0,5.91944


In [222]:
timeline['Timestamp']

0            0
1            0
2     0.256481
3     0.256481
4     0.358399
5     0.701681
6      3.01893
7      3.70405
8      3.84671
9      5.91944
10     5.91944
11     6.84421
12      7.2959
13     7.59557
14     7.59557
Name: Timestamp, dtype: object

In [223]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

8.033115437118887 9.232500014637587 8.033115437118887
New call arrived.
New call arrived. No more capacity. Reject call.


In [224]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

10.641308354526249 9.232500014637587 9.232500014637587
Now finish at-location treatment. Start going to hospital.


In [225]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.256481
3,1,1,1,0.256481
4,1,1,2,0.358399
5,0,0,2,0.701681
6,2,,0,3.01893
7,3,,0,3.70405
8,4,,0,3.84671
9,5,,0,5.91944
